# Automated ML


In [1]:
!pip install xgboost==0.90
!pip install pandas_profiling==2.9.0

In [2]:
import json
import sys
import os
import numpy as np
import pandas as pd
import shutil
import joblib
import requests

from sklearn.model_selection import train_test_split

#from TrainCovid19Infections import clean_data

from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.run import Run
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails

from azureml.train.automl import AutoMLConfig

from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.train.automl import constants
from azureml.automl.core.shared import constants
from pandas_profiling import ProfileReport

## Initialize Workspace

In [3]:
# Get current workspace from config
ws = Workspace.from_config()
    
ws.get_details()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

project_folder = './capstone-project'
experiment_name = 'Cov19InfectionAutoMlExperiment'
experiment=Experiment(ws, experiment_name)
experiment

Workspace name: wsptest
Azure region: eastus2
Subscription id: c04b3d3f-4994-454d-96ff-aa3f2050b57f
Resource group: testingmlfunctionnalities


Experiment(Name: Cov19InfectionAutoMlExperiment,
Workspace: wsptest)

## Create Cluster

Get cluster if it exists else create one

In [4]:
# Create compute cluster
cpu_cluster_name = "Covid19Cluster"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('A cluster with the same name already exists. If you are trying to create a new one please use a new cluster name')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4,identity_type="SystemAssigned")
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)
# Get a detailed status for the current cluster. 
print(cpu_cluster.get_status().serialize())

A cluster with the same name already exists. If you are trying to create a new one please use a new cluster name
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-26T01:37:42.127000+00:00', 'errors': None, 'creationTime': '2021-03-23T18:45:47.925166+00:00', 'modifiedTime': '2021-03-23T18:46:05.507627+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

### Overview

I Chose a COVID-19 World Wide infections Dataset that holds a track of Covid19's propagation rate, new cases, patients with chronic deseases and death rate per country. 

Since the covid-19 pandemic is among the hottest subjects in the world, and as a member of the society being interested in such statistics calculation can help further scientists or even regular people to better understand the global effect of this deadly virus all over the world.

I used Github to download the Dataset.

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Covid19InfectionsDataset"
description_text = "Covid19 Vaccination DataSet from Github"
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./github/owid-covid-data.csv'],
                       target_path ='train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)

if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    original_path = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
    ds = TabularDatasetFactory.from_delimited_files(original_path, infer_column_types=True, separator=',', header=True, index=False)
    
    #Register Dataset in Workspace
    dataset = ds.register(workspace=ws,
                          name=key,
                          description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

Uploading an estimated of 1 files
Uploading ./github/owid-covid-data.csv
Uploaded ./github/owid-covid-data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


total_cases      new_cases  new_cases_smoothed  total_deaths  \
count  7.590800e+04   75906.000000        74905.000000  6.656900e+04   
mean   6.933484e+05    5267.220799         5262.048175  2.021219e+04   
std    4.850087e+06   32675.802795        32254.835800  1.186788e+05   
min    1.000000e+00  -74347.000000        -6223.000000  1.000000e+00   
25%    7.620000e+02       2.000000            6.143000  4.000000e+01   
50%    8.610000e+03      62.000000           75.286000  2.650000e+02   
75%    9.238175e+04     695.000000          721.000000  2.567000e+03   
max    1.254917e+08  880902.000000       739564.429000  2.755210e+06   

         new_deaths  new_deaths_smoothed  total_cases_per_million  \
count  66727.000000         74905.000000             75495.000000   
mean     132.896039           117.110210              8290.103995   
std      725.870682           663.331248             16214.400983   
min    -1918.000000          -232.143000                 0.001000   
25%        0.000000             0.000000               161.791000   
50%        2.000000             1.143000              1137.508000   
75%       16.000000            12.571000              7797.146000   
max    17903.000000         14431.429000            151258.655000   

       new_cases_per_million  new_cases_smoothed_per_million  \
count           75493.000000                    74497.000000   
mean               68.937740                       68.699809   
std               171.161501                      144.339264   
min             -2153.437000                     -276.825000   
25%                 0.151000                        1.040000   
50%                 6.710000                        8.864000   
75%                59.754000                       67.241000   
max              8652.658000                     2648.773000   

       total_deaths_per_million  ...    median_age  aged_65_older  \
count              66169.000000  ...  70022.000000   69218.000000   
mean                 192.249212  ...     30.543811       8.787839   
std                  340.564356  ...      9.113683       6.232821   
min                    0.001000  ...     15.100000       1.144000   
25%                    5.903000  ...     22.200000       3.441000   
50%                   33.396000  ...     29.700000       6.378000   
75%                  204.623000  ...     39.100000      14.312000   
max                 2416.171000  ...     48.200000      27.049000   

       aged_70_older  gdp_per_capita  cardiovasc_death_rate  \
count   69628.000000    70280.000000           70884.000000   
mean        5.568556    19140.073723             257.336745   
std         4.256517    19763.369616             118.589511   
min         0.526000      661.240000              79.370000   
25%         2.043000     4466.507000             167.295000   
50%         3.871000    12951.839000             242.648000   
75%         8.678000    27216.445000             329.635000   
max        18.493000   116935.600000             724.417000   

       diabetes_prevalence  handwashing_facilities  \
count         71812.000000            35587.000000   
mean              7.802957               50.978244   
std               3.948980               31.766562   
min               0.990000                1.188000   
25%               5.290000               20.859000   
50%               7.110000               49.839000   
75%              10.080000               83.241000   
max              30.530000               98.999000   

       hospital_beds_per_thousand  life_expectancy  human_development_index  
count                64831.000000     73428.000000             70826.000000  
mean                     3.033356        73.151822                 0.727637  
std                      2.466140         7.550676                 0.150082  
min                      0.100000        53.280000                 0.394000  
25%                      1.300000        67.880000                 0.602000  
50%            

In [9]:
def clean_data(ds):
    x_df = ds.to_pandas_dataframe().fillna(0)
    print(len(x_df.columns)) #59
    y_df = x_df.pop("new_cases").apply(lambda s: 1 if s > 1 else 0)
    x_df['tested_units']= np.where(x_df.tests_units == 'tests performed', True, False)
    x_df['testing_units'] = x_df.pop("tested_units").apply(lambda s: 1 if s == True else 0)
    print(len(x_df.columns)) #59
    iso_codes = pd.get_dummies(x_df.iso_code, prefix="iso_code")
    continent = pd.get_dummies(x_df.continent, prefix="continent")
    x_df = x_df.join([iso_codes,continent])
    print(len(x_df.columns)) #281
    x_df.drop([
        "tests_units",
        "location",
        "date",
        "iso_code",
        "continent",
        "hosp_patients_per_million",
        "weekly_icu_admissions",
        "weekly_icu_admissions_per_million",
        "weekly_hosp_admissions",
        "weekly_hosp_admissions_per_million",
        "new_tests_smoothed_per_thousand",
        "new_tests_smoothed","new_tests_per_thousand",
        "new_cases_per_million",
        "new_cases_smoothed_per_million",
        "new_cases_smoothed",
        "people_vaccinated",
        "people_fully_vaccinated",
        "new_vaccinations",
        "new_vaccinations_smoothed",
        "total_vaccinations_per_hundred",
        "people_vaccinated_per_hundred",
        "people_fully_vaccinated_per_hundred",
        "new_vaccinations_smoothed_per_million",
        "total_vaccinations"],
         inplace=True, axis=1)
    print(len(x_df.columns))#259
    return x_df,y_df

In [10]:
# Use the clean_data function fro data cleaning:
x, y = clean_data(dataset)
data = pd.concat([x,y],axis=1)
data.head(10)

59
59
281
256


total_cases  total_deaths  new_deaths  new_deaths_smoothed  \
0          1.0           0.0         0.0                  0.0   
1          1.0           0.0         0.0                  0.0   
2          1.0           0.0         0.0                  0.0   
3          1.0           0.0         0.0                  0.0   
4          1.0           0.0         0.0                  0.0   
5          1.0           0.0         0.0                  0.0   
6          1.0           0.0         0.0                  0.0   
7          1.0           0.0         0.0                  0.0   
8          2.0           0.0         0.0                  0.0   
9          4.0           0.0         0.0                  0.0   

   total_cases_per_million  total_deaths_per_million  new_deaths_per_million  \
0                    0.026                       0.0                     0.0   
1                    0.026                       0.0                     0.0   
2                    0.026                       0.0                     0.0   
3                    0.026                       0.0                     0.0   
4                    0.026                       0.0                     0.0   
5                    0.026                       0.0                     0.0   
6                    0.026                       0.0                     0.0   
7                    0.026                       0.0                     0.0   
8                    0.051                       0.0                     0.0   
9                    0.103                       0.0                     0.0   

   new_deaths_smoothed_per_million  reproduction_rate icu_patients  ...  \
0                              0.0                0.0            0  ...   
1                              0.0                0.0            0  ...   
2                              0.0                0.0            0  ...   
3                              0.0                0.0            0  ...   
4                              0.0                0.0            0  ...   
5                              0.0                0.0            0  ...   
6                              0.0                0.0            0  ...   
7                              0.0                0.0            0  ...   
8                              0.0                0.0            0  ...   
9                              0.0                0.0            0  ...   

  iso_code_ZMB iso_code_ZWE continent_0 continent_Africa continent_Asia  \
0            0            0           0                0              1   
1            0            0           0                0              1   
2            0            0           0                0              1   
3            0            0           0                0              1   
4            0            0           0                0              1   
5            0            0           0                0              1   
6            0            0           0                0              1   
7            0            0           0                0              1   
8            0            0           0                0              1   
9            0            0           0                0              1   

  continent_Europe continent_North America  continent_Oceania  \
0                0                       0                  0   
1                0                       0                  0   
2                0                       0                  0   
3                0                       0                  0   
4                0                       0                  0   
5                0                       0                  0   
6                0                       0                  0   
7                0                       0                  0   
8                0                       0                  0   
9                0                       0                  0   

   continent_South America  new_cases 

In [ ]:
profile = ProfileReport(df, title="Cleaned Data Profiling", explorative=True)
profile.to_notebook_iframe()
profile.to_file("outputs/DataCleaningReport.html")

Summarize dataset:   0%|          | 0/73 [00:00<?, ?it/s]

In [15]:
# Split data into train and test sets.
training_data,validation_data = train_test_split(data,test_size = 0.5,random_state = 42,shuffle=True)
validation_data.head(10)

total_cases  new_cases_smoothed  total_deaths  new_deaths  \
46690          7.0               0.714           1.0         0.0   
8147        2557.0               8.714          41.0         0.0   
55587    1054273.0           11525.714       19861.0       502.0   
43936       1439.0              93.571          74.0         3.0   
33097    1424596.0            6472.857       58038.0        79.0   
65146     470973.0            8345.714       29152.0        58.0   
9093      244380.0             953.429       11508.0        38.0   
35384      45764.0            1361.429        1042.0         8.0   
40166     142671.0             869.143        2340.0        10.0   
30730          0.0               0.000           0.0         0.0   

       new_deaths_smoothed  total_cases_per_million  new_cases_per_million  \
46690                0.143                    0.190                  0.027   
8147                 0.000                  210.918                  0.000   
55587              445.000                27856.475                328.352   
43936                5.000                  309.485                 33.551   
33097               79.571                16960.899                 78.542   
65146               32.571                10073.258                173.565   
9093                33.429                20935.440                103.144   
35384                4.857                  361.838                 12.627   
40166               15.286                20763.359                156.157   
30730                0.000                    0.000                  0.000   

       new_cases_smoothed_per_million  total_deaths_per_million  \
46690                           0.019                     0.027   
8147                            0.719                     3.382   
55587                         304.538                   524.776   
43936                          20.124                    15.915   
33097                          77.064                   690.987   
65146                         178.500                   623.508   
9093                           81.678                   985.862   
35384                          10.764                     8.239   
40166                         126.489                   340.548   
30730                           0.000                     0.000   

       new_deaths_per_million  ...  iso_code_ZMB  iso_code_ZWE continent_0  \
46690                   0.000  ...             0             0           0   
8147                    0.000  ...             0             0           0   
55587                  13.264  ...             0             0           0   
43936                   0.645  ...             0             0           0   
33097                   0.941  ...             0             0           0   
65146                   1.241  ...             0             0           0   
9093                    3.255  ...             0             0           0   
35384                   0.063  ...             0             0           0   
40166                   1.455  ...             0             0           0   
30730                   0.000  ...             0             0           0   

      continent_Africa continent_Asia continent_Europe  \
46690                1              0                0   
8147                 1              0                0   
55587                0              0                1   
43936                1              0                0   
33097                0              1                0   
65146                0              0                1   
9093                 0              0                0   
35384                0              1                0   
40166                1              0                0   
30730                0              1                0   

      continent_North America continent_Oceania continent_South America  \
46690                       0                 0                       0   
8147         

In [17]:
# Create necessary folders
if "automl_training" not in os.listdir():
    os.mkdir("./automl_training")
if "data" not in os.listdir():
    os.mkdir("./data")
if "outputs" not in os.listdir():
    os.mkdir("./outputs")
if "training" not in os.listdir():
    os.mkdir("./training")
# store training_dataset into it using datastore
script_folder = './automl_training/'    
os.makedirs(script_folder, exist_ok=True)
shutil.copy('TrainCovid19Infections.py', script_folder)
project_folder = './pipeline-project'

## AutoML Configuration

The settings used below refers to a classification task within a number of settings chosen based on the existing workspace and cluster configuration restrictions.

The parameters used here are:

* `n_cross_validation = 5` : Since our dataset is small. We apply cross validation with 3 folds instead of train/validation data split.
* `primary_metric = 'accuracy'` : The primary metric parameter determines the metric to be used during model training for optimization. Accuracy primary metric is chosen for binary classification dataset.
* `enable_early_stopping = True` : Whether to enable early termination if the score is not improving in the short term.
* `experiment_timeout_hours = 1.0` : Maximum amount of time in hours that all iterations combined can take before the experiment terminates.
* `max_concurrent_iterations = 4` : To help manage child runs and when they can be performed, we match the number of maximum concurrent iterations of our experiment to the number of nodes in the cluster. So, we get a dedicated cluster per experiment.
* `task = 'classification'` : This specifies the experiment type as classification.
* `compute_target = cpu_cluster` : Azure Machine Learning Managed Compute is a managed service that enables the ability to train machine learning models on clusters of Azure virtual machines. Here compute target is set to cpu_cluster which is already defined with 'STANDARD_D2_V2' and maximum nodes equal to 4.
* `training_data = train_data` : This specifies the training data to be used in this experiment which is set to train_data which is a part of the dataset uploaded to the datastore.
* `label_column_name = 'new_cases'` : The target column here is set to DEATH_EVENT which has values 1 if the patient deceased or 0 if the patient survived.
* `featurization= 'auto'` : This indicates that as part of preprocessing, data guardrails and featurization steps are performed automatically.
* `model_explainability = True`: Whether to enable explaining the best AutoML model at the end of all AutoML training iterations.
* `path=project_folder`:
* `debug_log = "Covid_automl_errors.log"`: The log file to write debug information to. If not specified, 'automl.log' is used.

In [19]:
#convert the training dataset to a CSV file and store it under the training folder
training_data.to_csv('automl_training/training_data.csv', index=False)
                     show_progress = True)

datastore.upload(src_dir='automl_training/',target_path='data/',overwrite = True)
# Get the dataset from the data folder
training_dataset = TabularDatasetFactory.from_delimited_files(path=[(datastore,('data/training_data.csv'))])
td = training_dataset.take(10).to_pandas_dataframe()

Uploading an estimated of 2 files
Uploading automl_training/TrainCovid19Infections.py
Uploaded automl_training/TrainCovid19Infections.py, 1 files out of an estimated total of 2
Uploading automl_training/training_data.csv
Uploaded automl_training/training_data.csv, 2 files out of an estimated total of 2
Uploaded 2 files


In [22]:
# Configure Automl settings
automl_settings = {
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
    "enable_early_stopping": True,
    "experiment_timeout_hours": 1.0,
    "max_concurrent_iterations": 4,
}
automl_config = AutoMLConfig(task = 'classification',
                             compute_target = cpu_cluster,
                             training_data = training_dataset,
                             label_column_name = 'new_cases',
                             featurization= 'auto',
                             path=project_folder,
                             model_explainability=True,
                             debug_log = "Covid_automl_errors.log",
                             **automl_settings)

In [23]:
# Experiment Submission
tag = {"Covid19Infections": "Capstone project: Covid19 AutoML Experiment"}
remote_run = experiment.submit(automl_config,tags=tag, show_output=True)

Running on remote.
No run_configuration provided, running on Covid19Cluster with default configuration
Running on remote compute: Covid19Cluster
Parent Run ID: AutoML_909f336d-d72b-412a-be6b-48822da21efa

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:      

## Run Details

Based on features, parameters and built-in assumptions: some models' assumptions match the characteristics of the data, so we may get a good fit. However, when a model assumptions do not match the characteristics of the data, we may get a bad fit.

Using Automated Machine Learning, we can clearly deduce that _VotingEnsemble_ holds the Best metric 0.9991 among others like _MaxAbsScaler, LightGBM_ with a slight difference of 0.0001.


In [24]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_909f336d-d72b-412a-be6b-48822da21efa',
 'target': 'Covid19Cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-24T09:21:27.901891Z',
 'endTimeUtc': '2021-03-24T10:54:32.371061Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'Covid19Cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Cov19InfectionAutoMlExperiment","subscription_id":"c04b3d3f-4994-454d-96ff-aa3f2050b57f","resource_group":"testingmlfunctionnalities","workspace_name":"wsptest","region":"eastus2","compute_target":"Covid19Cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y

In [25]:
# List best models of HyperDrive Run and AutoML Run to compare the accuracy of the models and choose the best among them to deploy
for model in Model.list(ws):
    print(model.name)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print('\t',tag_name,':',tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print('\t',prop_name,':',prop)
    print("\n")

## Best Model

In [ ]:
explaining_model_run_id = remote_run.id
print(explaining_model_run_id)
explaining_model_run = Run(experiment=experiment, run_id=explaining_model_run_id)
explaining_model_run.wait_for_completion()

In [26]:
# Retrieve and save best model.
best_automl_run, best_automl_model = remote_run.get_output()

In [27]:
# Get the metrics of the best selected run
best_run_metrics = best_automl_run.get_metrics()

# Print all metrics of the best run model
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)
    
print(best_automl_model._final_estimator)

balanced_accuracy 0.9990008846407088
average_precision_score_weighted 0.999969072216558
precision_score_micro 0.9991929185108045
AUC_weighted 0.9999714485969312
f1_score_macro 0.9989551373812627
precision_score_macro 0.9989098170233598
accuracy 0.9991929185108045
f1_score_weighted 0.999192960365184
matthews_correlation 0.9979106235069866
f1_score_micro 0.9991929185108045
AUC_macro 0.9999714485969312
norm_macro_recall 0.9980017692814176
recall_score_macro 0.9990008846407088
precision_score_weighted 0.9991932726316282
log_loss 0.05273565879153332
average_precision_score_macro 0.9999497925240302
recall_score_micro 0.9991929185108045
average_precision_score_micro 0.9999764958325599
weighted_accuracy 0.9993127847418493
AUC_micro 0.9999761307332247
recall_score_weighted 0.9991929185108045
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_909f336d-d72b-412a-be6b-48822da21efa_30/confusion_matrix
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_909f336d-d72b-412a-be6b-48822da

In [28]:
best_automl_run

Run(Experiment: Cov19InfectionAutoMlExperiment,
Id: AutoML_909f336d-d72b-412a-be6b-48822da21efa_30,
Type: azureml.scriptrun,
Status: Completed)

In [29]:
joblib.dump(best_automl_model, filename="./outputs/automl-model.pkl")

['./outputs/automl-model.pkl']

In [30]:
arr = os.listdir('./outputs/')
print(arr)

['automl-model.pkl', 'model.joblib']


In [31]:
from azureml.core.model import Model
model = Model.list(ws)
print (model)

[]


In [32]:
# register the best model
best_model_registered = remote_run.register_model(
                                           
                                            model_name = "Covid19-automl-model",
                                            tags={'Area': "Pandemic", 'Type': "Classification",'Method of execution':'Auto ML'},
                                            )
#print(best_model.name, best_model.id, sep='\t')
print(best_model_registered)

Model(workspace=Workspace.create(name='wsptest', subscription_id='c04b3d3f-4994-454d-96ff-aa3f2050b57f', resource_group='testingmlfunctionnalities'), name=Covid19-automl-model, id=Covid19-automl-model:5, version=5, tags={'Area': 'Pandemic', 'Type': 'Classification', 'Method of execution': 'Auto ML'}, properties={})


## Model Deployment

As the best model coming from AutoML run has better accuracy than the one coming from the HyperDrive run, I deploy it in the cell below, register it, create an inference config and deploy the model as a web service.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [49]:
model = Model.register(model_path="outputs/automl-model.pkl",
                       model_name="best-trained-automl-model",
                       workspace = ws)
#environment = best_automl_run.get_environment()
#best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')
#best_automl_run.download_file(constants.CONDA_ENV_FILE_PATH, 'environment.yml')

env = Environment.get(workspace=ws, name="AzureML-AutoML")

inference_config = InferenceConfig(entry_script='score.py', environment=env)

deployment_config = AciWebservice.deploy_configuration(
                                                       cpu_cores=1,
                                                       memory_gb=1,
                                                       description='Covid19 new cases prediction',
                                                       auth_enabled=True,
                                                       enable_app_insights= True,
                                                       collect_model_data = True)

service = Model.deploy(workspace=ws,
                       name='aci-covid19-service',
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,
                       overwrite=True)

service.wait_for_deployment(show_output=True)
print(service.state)

Registering model best-trained-automl-model
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-03-24 12:30:57+00:00 Creating Container Registry if not exists.
2021-03-24 12:30:57+00:00 Registering the environment.
2021-03-24 12:30:58+00:00 Use the existing image.
2021-03-24 12:30:58+00:00 Generating deployment configuration.
2021-03-24 12:30:59+00:00 Submitting deployment to compute..
2021-03-24 12:31:03+00:00 Checking the status of deployment aci-covid19-service..
2021-03-24 12:36:11+00:00 Checking the status of inference endpoint aci-covid19-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [51]:
print("Key " + service.get_keys()[0])
print("Swagger URI : "+service.swagger_uri)
print("Scoring URI : "+service.scoring_uri)

Key 8IAbAQECc6dQYzKZ8SMLWNv1C7cuEdee
Swagger URI : http://245a1c75-e9bc-473a-b5a5-55887aa085d6.eastus2.azurecontainer.io/swagger.json
Scoring URI : http://245a1c75-e9bc-473a-b5a5-55887aa085d6.eastus2.azurecontainer.io/score


In [52]:
# testing the endpoint
x_test = validation_data.sample(2)
y_test = x_test['new_cases']
x_test.drop(['new_cases'], inplace=True, axis=1)

In [53]:
Covid19DataTesting= json.dumps({'data': x_test.to_dict(orient='records')})
print(Covid19DataTesting)

{"data": [{"total_cases": 2882.0, "new_cases_smoothed": 6.571, "total_deaths": 172.0, "new_deaths": 1.0, "new_deaths_smoothed": 1.0, "total_cases_per_million": 276.502, "new_cases_per_million": 0.384, "new_cases_smoothed_per_million": 0.63, "total_deaths_per_million": 16.502, "new_deaths_per_million": 0.096, "new_deaths_smoothed_per_million": 0.096, "reproduction_rate": 0.88, "icu_patients": 0, "icu_patients_per_million": 0, "hosp_patients": 0, "new_tests": "1074.0", "total_tests": "155037.0", "total_tests_per_thousand": "14.874", "positive_rate": "0.002", "tests_per_case": "507.7", "stringency_index": 68.52, "population": 10423056.0, "population_density": 83.479, "median_age": 45.3, "aged_65_older": 20.396, "aged_70_older": 14.524, "gdp_per_capita": 24574.382, "extreme_poverty": "1.5", "cardiovasc_death_rate": 175.695, "diabetes_prevalence": 4.55, "female_smokers": "35.3", "male_smokers": "52.0", "handwashing_facilities": 0.0, "hospital_beds_per_thousand": 4.21, "life_expectancy": 82.

In [54]:
headers = {'Content-type': 'application/json'}
headers['Authorization'] = f'Bearer {service.get_keys()[0]}'

# Make the request and display the response
response = requests.post(service.scoring_uri, Covid19DataTesting, headers=headers)
print('Prediction :', response.text)

# Print original labels
print('True Values :', y_test.values)

Prediction : [1, 1]
True Values : [1 1]


In [55]:
print(service.get_logs())

2021-03-24T12:35:20,702261600+00:00 - gunicorn/run 
2021-03-24T12:35:20,699224000+00:00 - rsyslog/run 
2021-03-24T12:35:20,709335100+00:00 - iot-server/run 
2021-03-24T12:35:20,740241400+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [41]:
#service.delete()
#cpu_cluster.delete()